In [1]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install simple_colors

In [3]:
import os
import math
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn, Tensor
import torch.utils.data as data_utils
from functools import partial, reduce
from simple_colors import *
import warnings
warnings.filterwarnings('ignore')
from metrics import *
from DataLoading import *
from utils.masking import *
from utils.metrics import *
from utils.tools import *
from utils.timefeatures import *
from utils.getmetrics import *
from Metrics import *
from RWLoading import *

In [ ]:
from Models.DLinear import *
from Models.Autoformer import *
from Models.FEDformer import *
from Models.FFNN import *
from Models.GRU import *
from Models.GRUAtt import *
from Models.Informer import *
from Models.LSTM import *
from Models.from Models.LSTMAtt import *
from Models.NLinear import *
from Models.PatchTST import *
from Models.RandomWalk import *
from Models.Seq2SeqGRU import *
from Models.Seq2SeqLSTM import *
from Models.Transformer import *

In [29]:
horizons = 4
test_ratio = 0.5
file_name = '/kaggle/input/pandemic/covid_weekly.csv'
features = ['date','new_deaths','icu_patients','hosp_patients','reproduction_rate','new_cases','new_tests','new_vaccinations']
epochs = 20
patience = 3
RNN_layers = 1
in_dim = len(features)-1
tf = False
random_seed = 2021
tf_ratio = 0.3
models = ["LSTM"]
RWhorizon = 4
batch_size =32

In [35]:
for learning_rate in [0.001]:
        for lags in [8]:
            for layers in [2]:
                for RNN_hidden_size in [256]:
                    for m in models:
                        print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size,green(' layers: ','bold'),layers,layers-1)
                        forecast_matrix = pd.DataFrame(columns = ['date'])
                        print(blue(m,'bold'))
                        random.seed(random_seed)
                        np.random.seed(random_seed)
                        torch.manual_seed(random_seed)
                        data, observed = data_processing(file_name, features)
                        scalers, df = data_transform_std(data, test_ratio)
                        obs_scalers, observed_df = data_transform_std(observed, 1.0)
                        x, y = make_input_output_sequences(data.values, lags, horizons, True)
                        x_train, x_test, y_train, y_test = train_test_split(x, y, test_ratio)
                        if(m == 'FFNN'):
                            learning_rate = 0.001
                            batch_size = 32
                            lags = 4
                            RNN_hidden_size = 512
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = FFNN(RNN_hidden_size,RNN_hidden_size, lags, horizons, len(features)-1).to(device)
                        elif(m == 'LSTM'):
                            learning_rate = 0.001
                            batch_size = 4
                            RNN_hidden_size = 128
                            lags = 4
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = LSTM(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif(m == 'GRU'):
                            learning_rate = 0.001
                            batch_size = 16
                            RNN_hidden_size = 128
                            lags = 4
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = GRU(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif(m == 'LSTM_Seq2Seq'):
                            learning_rate = 0.001
                            batch_size = 32
                            RNN_hidden_size = 128
                            lags = 4
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = LSTM_Seq2Seq(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif(m == 'GRU_Seq2Seq'):
                            learning_rate = 0.001
                            batch_size = 4
                            RNN_hidden_size = 64
                            lags = 4
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = GRU_Seq2Seq(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif(m == 'LSTM_Seq2Seq_Att'):
                            learning_rate = 0.001
                            batch_size = 8
                            RNN_hidden_size = 64
                            lags = 4
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = LSTM_Seq2Seq_Att(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif(m == 'GRU_Seq2Seq_Att'):
                            learning_rate = 0.001
                            batch_size = 16
                            RNN_hidden_size = 512
                            lags = 6
                            print(green('learning_rate: ','bold'),learning_rate, green(' lags: ','bold'),lags,green(' batch_size: ','bold')
                              ,batch_size, green(' IFmodel_dim: ','bold'),RNN_hidden_size)
                            model = GRU_Seq2Seq_Att(RNN_hidden_size, RNN_layers, lags, horizons, len(features)-1).to(device)
                        elif (m == "Nlinear"):
                            class Configs(object):
                                ab = 0
                                modes = 64
                                mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [12, 24]
                                L = 3
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = horizons
                                pred_len = horizons
                                output_attention = True
                                enc_in = in_dim
                                individual = True
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'd'
                                factor = 1
                                n_heads = 8
                                d_ff = 4
                                e_layers = 1
                                d_layers = 1
                                distil=False
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0

                            config = Configs()
                            model = Nlinear(config).to(device)
                        elif (m == "PatchTST"):
                            class Configs(object):
                                task_name = 'long_term_forecast'
                                ab = 0
                                modes = 64
                                mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [12, 24]
                                L = 3
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = horizons
                                pred_len = horizons
                                output_attention = True
                                enc_in = in_dim
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'h'
                                factor = 1
                                n_heads = 8
                                d_ff = RNN_hidden_size
                                e_layers = 4
                                d_layers = 4
                                distil=False
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0
                                

                            config = Configs()
                            model = PatchTST(config).to(device)
                        elif (m == "Transformer"):
                            class Configs(object):
                                ab = 0
                                modes = 64
    #                             mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [12,24]
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = horizons
                                pred_len = horizons
                                output_attention = False
                                enc_in = in_dim
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'h'
                                factor = 1
                                n_heads = 8
                                d_ff = 2048
                                e_layers = layers
                                d_layers = layers-1
                                distil=True
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0


                            config = Configs()
                            model = vTransformer(config).to(device)
                        elif (m == "Informer"):
                            class Configs(object):
                                ab = 0
                                modes = 64
    #                             mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [12,24]
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = horizons
                                pred_len = horizons
                                output_attention = False
                                enc_in = in_dim
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'h'
                                factor = 1
                                n_heads = 8
                                d_ff = 2048
                                e_layers = layers
                                d_layers = layers-1
                                distil=True
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0


                            config = Configs()
                            model = Informer(config).to(device)
                        elif (m == "Autoformer"):
                            class Configs(object):
                                ab = 0
                                modes = 64
                                mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [25, 25]
                                L = 3
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = horizons
                                pred_len = horizons
                                output_attention = True
                                enc_in = in_dim
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'w'
                                factor = 1
                                n_heads = 8
                                d_ff = 2048
                                e_layers = layers
                                d_layers = layers-1
                                distil=False
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0
                            config = Configs()
                            model = Autoformer(config).to(device)
                        elif (m == "FEDFormer"):
                            class Configs(object):
                                ab = 0
                                modes = 64
                                mode_select = 'random'
                                version = 'Fourier'
                                #version = 'Wavelets'
                                moving_avg = [25, 25]
                                L = 3
                                base = 'legendre'
                                cross_activation = 'tanh'
                                seq_len = lags
                                label_len = lags - horizons
                                pred_len = horizons
                                output_attention = True
                                enc_in = in_dim
                                dec_in = in_dim
                                d_model = RNN_hidden_size
                                embed = 'timeF'
                                dropout = 0.05
                                freq = 'd'
                                factor = 1
                                n_heads = 8
                                d_ff = 4
                                e_layers = 3
                                d_layers = 2
                                distil=False
                                c_out = in_dim
                                activation = 'gelu'
                                wavelet = 0

                            config = Configs()
                            model = FEDFormer(config).to(device)
                        smapeListPred = []
                        smapeListTrue = []
                        maePred=[]
                        maeTrue=[]
                        for w in range(x_test.shape[0]):
                            n_epochs_stop = patience
                            epochs_no_improve = 0
                            early_stop = False
                            min_val_loss = np.Inf
                   
                            x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
                            y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
                            x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
                            y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
                          
                            loss_fn = torch.nn.MSELoss()
                            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                            train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
                            train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)
                            lr = learning_rate
                            for epoch in range(epochs):
                                train_losses = []
                                model.train()
                                for x_t, y_t in train_loader:
                                    optimizer.zero_grad()
                                    y_pred = model(x_t, y_t, tf)
                                    loss = loss_fn(y_pred, y_t[:,:,0])
                                    loss.backward()
                                    optimizer.step()
                                    train_losses.append(loss.item())
                                train_loss = np.average(train_losses)
                                if train_loss < min_val_loss:
                                    epochs_no_improve = 0
                                    min_val_loss = train_loss
                                else:
                                    epochs_no_improve += 1
                                if epochs_no_improve == n_epochs_stop:
                                    break
          
                            with torch.no_grad():
                                model.eval()
                                y_test_pred = model(x_test_tensor, y_test_tensor, False)

                                smapeListTrue.append(scalers['scaler_new_deaths'].inverse_transform(y_test_tensor[:, :, 0].cpu().numpy())[0,:])
                                smapeListPred.append(scalers['scaler_new_deaths'].inverse_transform(y_test_pred.cpu().numpy())[0,:])
                                maeTrue.append(y_test_tensor[:, :, 0].cpu().numpy()[0,:])
                                maePred.append(y_test_pred.cpu().numpy()[0,:])
                            x_train, x_test, y_train, y_test = shift_sequence(x_train, y_train, x_test, y_test, 1, True)
                           
                        smapeListTrue = np.array(smapeListTrue)
                        smapeListPred = np.array(smapeListPred)
                        maeTrue = np.array(maeTrue)
                        maePred = np.array(maePred)

                        sMAPEmean=[smape(smapeListTrue[:,h], smapeListPred[:,h]) for h in range(4)]
                        print(sMAPEmean,np.mean(sMAPEmean))
                        MAEmean=[mae(smapeListTrue[:,h], smapeListPred[:,h]) for h in range(4)]
                        print(MAEmean,np.mean(MAEmean))
                        normMAEmean=[mae(maeTrue[:,h], maePred[:,h]) for h in range(4)]
                        print(normMAEmean,np.mean(normMAEmean))

learning_rate:  0.001  lags:  8  batch_size:  4  IFmodel_dim:  256  layers:  2 1
LSTM
learning_rate:  0.001  lags:  4  batch_size:  4  IFmodel_dim:  128
[14.790402183049842, 17.05072378810448, 18.791411194620252, 23.74427167675163] 18.59420221063155
[750.8900316455696, 868.1534810126582, 984.9107990506329, 1236.178303006329] 960.0331536787974
[0.12805492063111898, 0.14805277088020422, 0.16796425928043413, 0.2108148260961605] 0.16372169422197946
